In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import pathlib

In [3]:
python_path = pathlib.Path('.').absolute().parent/'python'
os.sys.path.insert(1, str(python_path))

In [4]:
import pinocchio as pin
from robot_properties_kuka.config import IiwaConfig

import meshcat
import meshcat.transformations as tf
import meshcat.geometry as g

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from tqdm import trange
import wandb

In [5]:
from vocam.qpnet import DataUtils, QPNet, QPNetObstacle
import vocam.qpnet

In [6]:
robot = IiwaConfig.buildRobotWrapper()
model, data = robot.model, robot.data
f_id = model.getFrameId("EE")

In [7]:
run = wandb.init(project="vocam", group="obstacle_train", entity="hjzhu")

wandb: Currently logged in as: hjzhu. Use `wandb login --relogin` to force relogin


In [8]:
wandb.config.nq = model.nq
wandb.config.nv = model.nv

wandb.config.n_col = 5
wandb.config.u_max = [2.5, 2.5, 2.5, 1.5, 1.5, 1.5, 1.0]
wandb.config.dt = 0.05

wandb.config.isvec = True
wandb.config.n_vars = 3 * model.nq * wandb.config.n_col + 2 * model.nq

wandb.config.lr_nn = 1e-3
wandb.config.batch_size = 64
wandb.config.epochs = 100

wandb.config.input_size = model.nq + model.nv + 3
wandb.config.obstacle_size = 3
wandb.config.output_size = 2 * wandb.config.n_vars

In [9]:
data_artifacts = run.use_artifact('hjzhu/vocam/data_obstacle:latest', type='dataset')
data_dir = data_artifacts.download()

wandb: Downloading large artifact data_obstacle:latest, 85.95MB. 20 files... Done. 0:0:0


In [10]:
data_train = torch.load(data_dir + "/data_obstacle_500_120.pt")
data_test = torch.load(data_dir + "/data_obstacle_20_119.pt")

In [11]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
network = QPNetObstacle(wandb.config.input_size, 
                        wandb.config.output_size,
                        wandb.config.obstacle_size).to(device)
optimizer = torch.optim.Adam(network.parameters(), lr=wandb.config.lr_nn)
criterion = torch.nn.L1Loss()
train_dataloader = DataLoader(data_train, batch_size=wandb.config.batch_size, 
                              shuffle=True)
test_dataloader = DataLoader(data_test, batch_size=wandb.config.batch_size, 
                              shuffle=True)
_ = wandb.watch(network)

In [12]:
for epoch in trange(wandb.config.epochs):
    loss_train = vocam.qpnet.train(network, criterion, optimizer, train_dataloader, device)
    loss_test = vocam.qpnet.test(network, criterion, test_dataloader, device)
    
    # Log the loss and accuracy values at the end of each epoch
    wandb.log({
        "Epoch": epoch,
        "Train Loss": loss_train,
        "Test Loss": loss_test})

100%|██████████| 100/100 [01:29<00:00,  1.12it/s]


In [13]:
# save trained model as artifact
n_run = run.name.split('-')[-1]
model_artifacts = wandb.Artifact('qpnet_obstacle_models', type="model")
model_dir = "../models/"
file_path = model_dir + 'qpnet_obstacle' + n_run + '.pt'
network.save(file_path)
model_artifacts.add_file(file_path)
run.log_artifact(model_artifacts)